In [8]:
# full_fol_resolution.py
from copy import deepcopy
from itertools import count

# ---------------------------
# Representation conventions:
# - Variables: strings starting with '?' e.g. '?x'
# - Constants: plain strings like 'a', 'John'
# - Predicates/Functions: tuples, e.g. ('P', arg1, arg2)
# - Negation: ('not', <literal>)
# - Connectives: ('and', A, B, ...), ('or', A, B, ...), ('implies', A, B), ('iff', A, B)
# - Quantifiers: ('forall', '?x', body), ('exists', '?x', body)
# ---------------------------

# --- Utilities ---
def is_variable(x):
    return isinstance(x, str) and x.startswith('?')

def is_compound(x):
    return isinstance(x, tuple) and len(x) > 0 and isinstance(x[0], str)

def fresh_name(prefix='v'):
    # generator for fresh names
    for i in count(1):
        yield f"{prefix}{i}"

_fresh_var_gen = fresh_name('V')

def fresh_var():
    return '?' + next(_fresh_var_gen)

# --- Formula transformers ---
def eliminate_iff(formula):
    if not is_compound(formula):
        return formula
    op = formula[0]
    if op == 'iff':
        A = eliminate_iff(formula[1])
        B = eliminate_iff(formula[2])
        # (A ↔ B)  -> ((A → B) ∧ (B → A))
        return ('and', ('implies', A, B), ('implies', B, A))
    else:
        return tuple([op] + [eliminate_iff(arg) for arg in formula[1:]])

def eliminate_implies(formula):
    if not is_compound(formula):
        return formula
    op = formula[0]
    if op == 'implies':
        A = eliminate_implies(formula[1])
        B = eliminate_implies(formula[2])
        # A → B  ==  ¬A ∨ B
        return ('or', ('not', A), B)
    else:
        return tuple([op] + [eliminate_implies(arg) for arg in formula[1:]])

def move_not_inwards(formula):
    # assumes no implies/iff remain
    if not is_compound(formula):
        return formula
    op = formula[0]
    if op == 'not':
        neg = formula[1]
        if not is_compound(neg):
            return ('not', neg)
        nop = neg[0]
        if nop == 'not':
            return move_not_inwards(neg[1])  # ~~A -> A
        if nop == 'and':
            # ¬(A ∧ B) -> ¬A ∨ ¬B
            return ('or',) + tuple(move_not_inwards(('not', a)) for a in neg[1:])
        if nop == 'or':
            # ¬(A ∨ B) -> ¬A ∧ ¬B
            return ('and',) + tuple(move_not_inwards(('not', a)) for a in neg[1:])
        if nop == 'forall':
            # ¬∀x A -> ∃x ¬A
            return ('exists', neg[1], move_not_inwards(('not', neg[2])))
        if nop == 'exists':
            # ¬∃x A -> ∀x ¬A
            return ('forall', neg[1], move_not_inwards(('not', neg[2])))
        # predicate negation
        return ('not', tuple([nop] + [move_not_inwards(a) for a in neg[1:]]))
    elif op in ('and', 'or'):
        return tuple([op] + [move_not_inwards(arg) for arg in formula[1:]])
    elif op in ('forall', 'exists'):
        return (op, formula[1], move_not_inwards(formula[2]))
    else:
        # atomic/predicate
        return tuple([op] + [move_not_inwards(arg) if is_compound(arg) else arg for arg in formula[1:]])

# Standardize variables apart (rename bound variables to unique ones)
def standardize_apart(formula, mapping=None):
    if mapping is None:
        mapping = {}
    if not is_compound(formula):
        if isinstance(formula, str) and formula in mapping:
            return mapping[formula]
        return formula
    op = formula[0]
    if op in ('forall', 'exists'):
        var = formula[1]
        newvar = fresh_var()
        mapping2 = mapping.copy()
        mapping2[var] = newvar
        return (op, newvar, standardize_apart(formula[2], mapping2))
    else:
        return tuple([op] + [standardize_apart(arg, mapping) for arg in formula[1:]])

# Skolemization: remove existential quantifiers by introducing skolem constants/functions
_skolem_count = count(1)
def new_skolem(name_hint='SK'):
    return f"{name_hint}{next(_skolem_count)}"

def skolemize(formula, universal_context=None):
    if universal_context is None:
        universal_context = []
    if not is_compound(formula):
        return formula
    op = formula[0]
    if op == 'forall':
        var = formula[1]
        return ('forall', var, skolemize(formula[2], universal_context + [var]))
    if op == 'exists':
        var = formula[1]
        # build a Skolem function or constant name
        if universal_context:
            # Skolem function of the universal_context variables
            func_name = new_skolem('F')
            def make_skolem_term():
                # represent function as ('f_name', arg1, ...)
                return tuple([func_name] + list(universal_context))
            sk_term = make_skolem_term()
        else:
            # Skolem constant
            sk_term = new_skolem('C')
        # replace occurrences of var in body with sk_term
        body = substitute(formula[2], {var: sk_term})
        return skolemize(body, universal_context)
    if op in ('and', 'or'):
        return (op,) + tuple(skolemize(arg, universal_context) for arg in formula[1:])
    if op == 'not':
        return ('not', skolemize(formula[1], universal_context))
    # atomic
    return tuple([op] + [skolemize(arg, universal_context) if is_compound(arg) else substitute(arg, {}) for arg in formula[1:]])

def substitute(term, subs):
    # subs maps variable names (like '?x') to terms (constants or compound)
    if isinstance(term, str):
        return subs.get(term, term)
    if is_compound(term):
        return tuple([term[0]] + [substitute(t, subs) for t in term[1:]])
    return term

# Drop universal quantifiers (after skolemization everything quantified is universal)
def drop_universal_quantifiers(formula):
    if not is_compound(formula):
        return formula
    op = formula[0]
    if op == 'forall':
        return drop_universal_quantifiers(formula[2])
    if op in ('and', 'or'):
        return (op,) + tuple(drop_universal_quantifiers(arg) for arg in formula[1:])
    if op == 'not':
        return ('not', drop_universal_quantifiers(formula[1]))
    return tuple([op] + [drop_universal_quantifiers(arg) for arg in formula[1:]])

# Distribute OR over AND to get CNF
def distribute_or_over_and(formula):
    # formula assumed to contain only 'and', 'or', 'not', atomic predicates
    if not is_compound(formula):
        return formula
    op = formula[0]
    if op == 'or':
        # flatten
        args = []
        for a in formula[1:]:
            da = distribute_or_over_and(a)
            if is_compound(da) and da[0] == 'or':
                args.extend(da[1:])
            else:
                args.append(da)
        # now if any arg is an 'and', distribute
        for i, a in enumerate(args):
            if is_compound(a) and a[0] == 'and':
                rest = args[:i] + args[i+1:]
                # (A ∨ (B ∧ C)) -> (A ∨ B) ∧ (A ∨ C)
                distributed_parts = []
                for conjunct in a[1:]:
                    distributed_parts.append(distribute_or_over_and(('or',) + tuple(rest + [conjunct])))
                return distribute_or_over_and(('and',) + tuple(distributed_parts))
        return ('or',) + tuple(args)
    if op == 'and':
        return ('and',) + tuple(distribute_or_over_and(arg) for arg in formula[1:])
    if op == 'not':
        return ('not',) + (distribute_or_over_and(formula[1]),)
    # atom
    return tuple([op] + [distribute_or_over_and(a) if is_compound(a) else a for a in formula[1:]])

# Flatten and extract clauses from CNF
def cnf_to_clauses(formula):
    # Returns list of clauses, each clause is a set of literals (literal: atomic tuple or ('not', atomic))
    if not is_compound(formula):
        return [frozenset([formula])]
    if formula[0] == 'and':
        clauses = []
        for arg in formula[1:]:
            clauses.extend(cnf_to_clauses(arg))
        return clauses
    if formula[0] == 'or':
        lits = set()
        for arg in formula[1:]:
            # arg might be ('not', atom) or atom
            if is_compound(arg) and arg[0] == 'or':
                # flatten nested OR
                for inner in cnf_to_clauses(arg):
                    # inner should be single clause
                    lits.update(set(inner))
            else:
                lits.add(arg)
        return [frozenset(lits)]
    # single literal (possibly negated)
    return [frozenset([formula])]

# --- Full pipeline to CNF ---
def to_cnf(formula):
    f = deepcopy(formula)
    f = eliminate_iff(f)
    f = eliminate_implies(f)
    f = move_not_inwards(f)
    f = standardize_apart(f)
    f = skolemize(f)
    f = drop_universal_quantifiers(f)
    f = distribute_or_over_and(f)
    return f

# --- Unification (works with variables ?x and compound terms) ---
def unify(x, y, theta=None):
    if theta is False:
        return False
    if theta is None:
        theta = {}
    if x == y:
        return theta
    if is_variable(x):
        return unify_var(x, y, theta)
    if is_variable(y):
        return unify_var(y, x, theta)
    if is_compound(x) and is_compound(y) and x[0] == y[0] and len(x) == len(y):
        for xi, yi in zip(x[1:], y[1:]):
            theta = unify(apply_substitution(xi, theta), apply_substitution(yi, theta), theta)
            if theta is False:
                return False
        return theta
    return False

def unify_var(var, x, theta):
    if var in theta:
        return unify(theta[var], x, theta)
    if is_variable(x) and x in theta:
        return unify(var, theta[x], theta)
    # occurs check: avoid var appearing in x
    if occurs_check(var, x, theta):
        return False
    theta2 = theta.copy()
    theta2[var] = x
    return theta2

def occurs_check(var, x, theta):
    # check if var occurs in x after applying theta
    x = apply_substitution(x, theta)
    if var == x:
        return True
    if is_compound(x):
        return any(occurs_check(var, xi, theta) for xi in x[1:])
    return False

def apply_substitution(literal, theta):
    if isinstance(literal, str):
        return theta.get(literal, literal)
    if is_compound(literal):
        return tuple([literal[0]] + [apply_substitution(arg, theta) for arg in literal[1:]])
    return literal

# --- Resolution ---
def negate_literal(lit):
    if is_compound(lit) and lit[0] == 'not':
        return lit[1]
    else:
        return ('not', lit)

def resolve(ci, cj):
    resolvents = []
    for di in ci:
        for dj in cj:
            # attempt to unify di with negation of dj
            theta = unify(di, negate_literal(dj))
            if theta is not False:
                new_clause = (set(ci) - {di}) | (set(cj) - {dj})
                new_clause = set(apply_substitution(l, theta) for l in new_clause)
                resolvents.append(new_clause)
            # also try unify neg(di) with dj
            theta2 = unify(negate_literal(di), dj)
            if theta2 is not False:
                new_clause = (set(ci) - {di}) | (set(cj) - {dj})
                new_clause = set(apply_substitution(l, theta2) for l in new_clause)
                resolvents.append(new_clause)
    return resolvents

def fol_resolution(clauses, verbose=True, max_iterations=10000):
    # clauses: list of sets of literals (literal is tuple or ('not', tuple))
    new = set()
    clauses = [set(c) for c in clauses]
    iteration = 0
    while True:
        iteration += 1
        if iteration > max_iterations:
            if verbose:
                print("Max iterations reached; stopping.")
            return False
        n = len(clauses)
        pairs = [(clauses[i], clauses[j]) for i in range(n) for j in range(i+1, n)]
        added_any = False
        for ci, cj in pairs:
            resolvents = resolve(ci, cj)
            for res in resolvents:
                if not res:
                    if verbose:
                        print("\nDerived empty clause. Goal is proved!")
                    return True
                fz = frozenset(res)
                if fz not in new and res not in clauses:
                    new.add(fz)
                    added_any = True
        if not added_any:
            if verbose:
                print("\nNo new clauses can be derived. Goal cannot be proved.")
            return False
        # move new into clauses
        for fz in new:
            if set(fz) not in clauses:
                clauses.append(set(fz))
        new.clear()

# --- Helper to pretty-print clauses ---
def print_clauses(clauses):
    for i, c in enumerate(clauses, 1):
        print(f"C{i}: {c}")


# --- Example usage ---
if __name__ == "__main__":

    tests = [

      {
        "name": "(Provable)",
        # KB:
        #   1. ∀x (P(x) → Q(x))
        #   2. P(a)
        # Goal:
        #   Q(a)
        "kb": [
            ('forall', '?x', ('implies', ('P', '?x'), ('Q', '?x'))),
            ('P', 'a')
        ],
        "goal": ('Q', 'a')
    },
      {
    "name": "(Not Provable)",
    # KB:
    #   1. ∀x (P(x) → Q(x))
    #   2. P(a)
    # Goal:
    #   Q(a)
    "kb": [
        ('forall', '?x', ('implies', ('P', '?x'), ('Q', '?x'))),
        ('P', 'a')
    ],
    "goal": ('R', 'a')
},



  ]

    for test in tests:
        print("\n===============================")
        print(test["name"])
        print("===============================\n")

        kb = test["kb"]
        goal = test["goal"]

        all_clauses = []
        for s in kb:
            cnf = to_cnf(s)
            clauses = cnf_to_clauses(cnf)
            all_clauses.extend([set(cl) for cl in clauses])

        clauses_for_proof = deepcopy(all_clauses)
        clauses_for_proof.append(set([('not', goal)]))

        print("CNF CLAUSES:")
        print_clauses(clauses_for_proof)

        print("\n--- Resolution ---")
        proved = fol_resolution(clauses_for_proof, verbose=True)
        print("\nRESULT:", proved)


(Provable)

CNF CLAUSES:
C1: {('not', ('P', '?V1')), ('Q', '?V1')}
C2: {('P', 'a')}
C3: {('not', ('Q', 'a'))}

--- Resolution ---

Derived empty clause. Goal is proved!

RESULT: True

(Not Provable)

CNF CLAUSES:
C1: {('not', ('P', '?V2')), ('Q', '?V2')}
C2: {('P', 'a')}
C3: {('not', ('R', 'a'))}

--- Resolution ---

No new clauses can be derived. Goal cannot be proved.

RESULT: False


In [9]:
# full_fol_resolution.py
from copy import deepcopy
from itertools import count

# ---------------------------
# Representation conventions:
# - Variables: strings starting with '?' e.g. '?x'
# - Constants: plain strings like 'a', 'John'
# - Predicates/Functions: tuples, e.g. ('P', arg1, arg2)
# - Negation: ('not', <literal>)
# - Connectives: ('and', A, B, ...), ('or', A, B, ...), ('implies', A, B), ('iff', A, B)
# - Quantifiers: ('forall', '?x', body), ('exists', '?x', body)
# ---------------------------

# --- Utilities ---
def is_variable(x):
    return isinstance(x, str) and x.startswith('?')

def is_compound(x):
    return isinstance(x, tuple) and len(x) > 0 and isinstance(x[0], str)

def fresh_name(prefix='v'):
    # generator for fresh names
    for i in count(1):
        yield f"{prefix}{i}"

_fresh_var_gen = fresh_name('V')

def fresh_var():
    return '?' + next(_fresh_var_gen)

# --- Formula transformers ---
def eliminate_iff(formula):
    if not is_compound(formula):
        return formula
    op = formula[0]
    if op == 'iff':
        A = eliminate_iff(formula[1])
        B = eliminate_iff(formula[2])
        # (A ↔ B)  -> ((A → B) ∧ (B → A))
        return ('and', ('implies', A, B), ('implies', B, A))
    else:
        return tuple([op] + [eliminate_iff(arg) for arg in formula[1:]])

def eliminate_implies(formula):
    if not is_compound(formula):
        return formula
    op = formula[0]
    if op == 'implies':
        A = eliminate_implies(formula[1])
        B = eliminate_implies(formula[2])
        # A → B  ==  ¬A ∨ B
        return ('or', ('not', A), B)
    else:
        return tuple([op] + [eliminate_implies(arg) for arg in formula[1:]])

def move_not_inwards(formula):
    # assumes no implies/iff remain
    if not is_compound(formula):
        return formula
    op = formula[0]
    if op == 'not':
        neg = formula[1]
        if not is_compound(neg):
            return ('not', neg)
        nop = neg[0]
        if nop == 'not':
            return move_not_inwards(neg[1])  # ~~A -> A
        if nop == 'and':
            # ¬(A ∧ B) -> ¬A ∨ ¬B
            return ('or',) + tuple(move_not_inwards(('not', a)) for a in neg[1:])
        if nop == 'or':
            # ¬(A ∨ B) -> ¬A ∧ ¬B
            return ('and',) + tuple(move_not_inwards(('not', a)) for a in neg[1:])
        if nop == 'forall':
            # ¬∀x A -> ∃x ¬A
            return ('exists', neg[1], move_not_inwards(('not', neg[2])))
        if nop == 'exists':
            # ¬∃x A -> ∀x ¬A
            return ('forall', neg[1], move_not_inwards(('not', neg[2])))
        # predicate negation
        return ('not', tuple([nop] + [move_not_inwards(a) for a in neg[1:]]))
    elif op in ('and', 'or'):
        return tuple([op] + [move_not_inwards(arg) for arg in formula[1:]])
    elif op in ('forall', 'exists'):
        return (op, formula[1], move_not_inwards(formula[2]))
    else:
        # atomic/predicate
        return tuple([op] + [move_not_inwards(arg) if is_compound(arg) else arg for arg in formula[1:]])

# Standardize variables apart (rename bound variables to unique ones)
def standardize_apart(formula, mapping=None):
    if mapping is None:
        mapping = {}
    if not is_compound(formula):
        if isinstance(formula, str) and formula in mapping:
            return mapping[formula]
        return formula
    op = formula[0]
    if op in ('forall', 'exists'):
        var = formula[1]
        newvar = fresh_var()
        mapping2 = mapping.copy()
        mapping2[var] = newvar
        return (op, newvar, standardize_apart(formula[2], mapping2))
    else:
        return tuple([op] + [standardize_apart(arg, mapping) for arg in formula[1:]])

# Skolemization: remove existential quantifiers by introducing skolem constants/functions
_skolem_count = count(1)
def new_skolem(name_hint='SK'):
    return f"{name_hint}{next(_skolem_count)}"

def skolemize(formula, universal_context=None):
    if universal_context is None:
        universal_context = []
    if not is_compound(formula):
        return formula
    op = formula[0]
    if op == 'forall':
        var = formula[1]
        return ('forall', var, skolemize(formula[2], universal_context + [var]))
    if op == 'exists':
        var = formula[1]
        # build a Skolem function or constant name
        if universal_context:
            # Skolem function of the universal_context variables
            func_name = new_skolem('F')
            def make_skolem_term():
                # represent function as ('f_name', arg1, ...)
                return tuple([func_name] + list(universal_context))
            sk_term = make_skolem_term()
        else:
            # Skolem constant
            sk_term = new_skolem('C')
        # replace occurrences of var in body with sk_term
        body = substitute(formula[2], {var: sk_term})
        return skolemize(body, universal_context)
    if op in ('and', 'or'):
        return (op,) + tuple(skolemize(arg, universal_context) for arg in formula[1:])
    if op == 'not':
        return ('not', skolemize(formula[1], universal_context))
    # atomic
    return tuple([op] + [skolemize(arg, universal_context) if is_compound(arg) else substitute(arg, {}) for arg in formula[1:]])

def substitute(term, subs):
    # subs maps variable names (like '?x') to terms (constants or compound)
    if isinstance(term, str):
        return subs.get(term, term)
    if is_compound(term):
        return tuple([term[0]] + [substitute(t, subs) for t in term[1:]])
    return term

# Drop universal quantifiers (after skolemization everything quantified is universal)
def drop_universal_quantifiers(formula):
    if not is_compound(formula):
        return formula
    op = formula[0]
    if op == 'forall':
        return drop_universal_quantifiers(formula[2])
    if op in ('and', 'or'):
        return (op,) + tuple(drop_universal_quantifiers(arg) for arg in formula[1:])
    if op == 'not':
        return ('not', drop_universal_quantifiers(formula[1]))
    return tuple([op] + [drop_universal_quantifiers(arg) for arg in formula[1:]])

# Distribute OR over AND to get CNF
def distribute_or_over_and(formula):
    # formula assumed to contain only 'and', 'or', 'not', atomic predicates
    if not is_compound(formula):
        return formula
    op = formula[0]
    if op == 'or':
        # flatten
        args = []
        for a in formula[1:]:
            da = distribute_or_over_and(a)
            if is_compound(da) and da[0] == 'or':
                args.extend(da[1:])
            else:
                args.append(da)
        # now if any arg is an 'and', distribute
        for i, a in enumerate(args):
            if is_compound(a) and a[0] == 'and':
                rest = args[:i] + args[i+1:]
                # (A ∨ (B ∧ C)) -> (A ∨ B) ∧ (A ∨ C)
                distributed_parts = []
                for conjunct in a[1:]:
                    distributed_parts.append(distribute_or_over_and(('or',) + tuple(rest + [conjunct])))
                return distribute_or_over_and(('and',) + tuple(distributed_parts))
        return ('or',) + tuple(args)
    if op == 'and':
        return ('and',) + tuple(distribute_or_over_and(arg) for arg in formula[1:])
    if op == 'not':
        return ('not',) + (distribute_or_over_and(formula[1]),)
    # atom
    return tuple([op] + [distribute_or_over_and(a) if is_compound(a) else a for a in formula[1:]])

# Flatten and extract clauses from CNF
def cnf_to_clauses(formula):
    # Returns list of clauses, each clause is a set of literals (literal: atomic tuple or ('not', atomic))
    if not is_compound(formula):
        return [frozenset([formula])]
    if formula[0] == 'and':
        clauses = []
        for arg in formula[1:]:
            clauses.extend(cnf_to_clauses(arg))
        return clauses
    if formula[0] == 'or':
        lits = set()
        for arg in formula[1:]:
            # arg might be ('not', atom) or atom
            if is_compound(arg) and arg[0] == 'or':
                # flatten nested OR
                for inner in cnf_to_clauses(arg):
                    # inner should be single clause
                    lits.update(set(inner))
            else:
                lits.add(arg)
        return [frozenset(lits)]
    # single literal (possibly negated)
    return [frozenset([formula])]

# --- Full pipeline to CNF ---
def to_cnf(formula):
    f = deepcopy(formula)
    f = eliminate_iff(f)
    f = eliminate_implies(f)
    f = move_not_inwards(f)
    f = standardize_apart(f)
    f = skolemize(f)
    f = drop_universal_quantifiers(f)
    f = distribute_or_over_and(f)
    return f

# --- Unification (works with variables ?x and compound terms) ---
def unify(x, y, theta=None):
    if theta is False:
        return False
    if theta is None:
        theta = {}
    if x == y:
        return theta
    if is_variable(x):
        return unify_var(x, y, theta)
    if is_variable(y):
        return unify_var(y, x, theta)
    if is_compound(x) and is_compound(y) and x[0] == y[0] and len(x) == len(y):
        for xi, yi in zip(x[1:], y[1:]):
            theta = unify(apply_substitution(xi, theta), apply_substitution(yi, theta), theta)
            if theta is False:
                return False
        return theta
    return False

def unify_var(var, x, theta):
    if var in theta:
        return unify(theta[var], x, theta)
    if is_variable(x) and x in theta:
        return unify(var, theta[x], theta)
    # occurs check: avoid var appearing in x
    if occurs_check(var, x, theta):
        return False
    theta2 = theta.copy()
    theta2[var] = x
    return theta2

def occurs_check(var, x, theta):
    # check if var occurs in x after applying theta
    x = apply_substitution(x, theta)
    if var == x:
        return True
    if is_compound(x):
        return any(occurs_check(var, xi, theta) for xi in x[1:])
    return False

def apply_substitution(literal, theta):
    if isinstance(literal, str):
        return theta.get(literal, literal)
    if is_compound(literal):
        return tuple([literal[0]] + [apply_substitution(arg, theta) for arg in literal[1:]])
    return literal

# --- Resolution ---
def negate_literal(lit):
    if is_compound(lit) and lit[0] == 'not':
        return lit[1]
    else:
        return ('not', lit)

def resolve(ci, cj):
    resolvents = []
    for di in ci:
        for dj in cj:
            # attempt to unify di with negation of dj
            theta = unify(di, negate_literal(dj))
            if theta is not False:
                new_clause = (set(ci) - {di}) | (set(cj) - {dj})
                new_clause = set(apply_substitution(l, theta) for l in new_clause)
                resolvents.append(new_clause)
            # also try unify neg(di) with dj
            theta2 = unify(negate_literal(di), dj)
            if theta2 is not False:
                new_clause = (set(ci) - {di}) | (set(cj) - {dj})
                new_clause = set(apply_substitution(l, theta2) for l in new_clause)
                resolvents.append(new_clause)
    return resolvents

def fol_resolution(clauses, verbose=True, max_iterations=10000):
    # clauses: list of sets of literals (literal is tuple or ('not', tuple))
    new = set()
    clauses = [set(c) for c in clauses]
    iteration = 0
    while True:
        iteration += 1
        if iteration > max_iterations:
            if verbose:
                print("Max iterations reached; stopping.")
            return False
        n = len(clauses)
        pairs = [(clauses[i], clauses[j]) for i in range(n) for j in range(i+1, n)]
        added_any = False
        for ci, cj in pairs:
            resolvents = resolve(ci, cj)
            for res in resolvents:
                if not res:
                    if verbose:
                        print("\nDerived empty clause. Goal is proved!")
                    return True
                fz = frozenset(res)
                if fz not in new and res not in clauses:
                    new.add(fz)
                    added_any = True
        if not added_any:
            if verbose:
                print("\nNo new clauses can be derived. Goal cannot be proved.")
            return False
        # move new into clauses
        for fz in new:
            if set(fz) not in clauses:
                clauses.append(set(fz))
        new.clear()

# --- Helper to pretty-print clauses ---
def print_clauses(clauses):
    for i, c in enumerate(clauses, 1):
        print(f"C{i}: {c}")


# --- Example usage ---
if __name__ == "__main__":

    tests = [

      {
          "name": "Example 1 (Provable)",
          # KB:
          #   1. ∀x (Human(x) → Mortal(x))
          #   2. Human(Socrates)
          # Goal:
          #   Mortal(Socrates)
          "kb": [
              ('forall', '?x', ('implies', ('Human', '?x'), ('Mortal', '?x'))),
              ('Human', 'Socrates')
          ],
          "goal": ('Mortal', 'Socrates')
      },

      {
          "name": "Example 2 (Provable)",
          # KB:
          #   1. ∀x (Parent(x) → Ancestor(x))
          #   2. ∀x∀y (Ancestor(x) → Ancestor(F1(x), y))
          #   3. Parent(Alice)
          # Goal:
          #   Ancestor(Alice)
          "kb": [
              ('forall', '?x', ('implies', ('Parent', '?x'), ('Ancestor', '?x'))),
              ('forall', '?x', ('forall', '?y',
                  ('implies', ('Ancestor', '?x'), ('Ancestor', ('F1', '?x'), '?y'))
              )),
              ('Parent', 'Alice')
          ],
          "goal": ('Ancestor', 'Alice')
      },

      {
          "name": "Example 3 (NOT provable — should fail)",
          # KB:
          #   1. ∀x (P(x) → Q(x))
          #   2. ∀x (R(x) → S(x))
          #   3. P(a)
          # Goal:
          #   T(a)
          "kb": [
              ('forall', '?x', ('implies', ('P', '?x'), ('Q', '?x'))),
              ('forall', '?x', ('implies', ('R', '?x'), ('S', '?x'))),
              ('P', 'a')
          ],
          "goal": ('T', 'a')
      }

  ]

    for test in tests:
        print("\n===============================")
        print(test["name"])
        print("===============================\n")

        kb = test["kb"]
        goal = test["goal"]

        all_clauses = []
        for s in kb:
            cnf = to_cnf(s)
            clauses = cnf_to_clauses(cnf)
            all_clauses.extend([set(cl) for cl in clauses])

        clauses_for_proof = deepcopy(all_clauses)
        clauses_for_proof.append(set([('not', goal)]))

        print("CNF CLAUSES:")
        print_clauses(clauses_for_proof)

        print("\n--- Resolution ---")
        proved = fol_resolution(clauses_for_proof, verbose=True)
        print("\nRESULT:", proved)


Example 1 (Provable)

CNF CLAUSES:
C1: {('Mortal', '?V1'), ('not', ('Human', '?V1'))}
C2: {('Human', 'Socrates')}
C3: {('not', ('Mortal', 'Socrates'))}

--- Resolution ---

Derived empty clause. Goal is proved!

RESULT: True

Example 2 (Provable)

CNF CLAUSES:
C1: {('not', ('Parent', '?V2')), ('Ancestor', '?V2')}
C2: {('Ancestor', ('F1', '?V3'), '?V4'), ('not', ('Ancestor', '?V3'))}
C3: {('Parent', 'Alice')}
C4: {('not', ('Ancestor', 'Alice'))}

--- Resolution ---

Derived empty clause. Goal is proved!

RESULT: True

Example 3 (NOT provable — should fail)

CNF CLAUSES:
C1: {('Q', '?V5'), ('not', ('P', '?V5'))}
C2: {('S', '?V6'), ('not', ('R', '?V6'))}
C3: {('P', 'a')}
C4: {('not', ('T', 'a'))}

--- Resolution ---

No new clauses can be derived. Goal cannot be proved.

RESULT: False
